**Incomplete project**

In [128]:
import pandas as pd
import numpy as np

In [129]:
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [130]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [131]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


# Processing Basics Data

In [132]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8941339 entries, 0 to 8941338
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 614.0+ MB


In [133]:
basics.duplicated().sum()

0

In [134]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64

In [135]:
# replacing all '\N' values with np.nan
basics = basics.replace({'\\N':np.nan})
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1192070
endYear           8850464
runtimeMinutes    6535931
genres             408443
dtype: int64

In [136]:
# Eliminating movies that are null for 'runtimeMinutes' and 'genre'
basics = basics.dropna(subset=['runtimeMinutes','genres'])
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear           34196
endYear           2295334
runtimeMinutes          0
genres                  0
dtype: int64

In [137]:
# Excluding movies without the titleType movie 
basics = basics[basics['titleType'] == 'movie']
basics['titleType'].value_counts()

movie    360845
Name: titleType, dtype: int64

In [138]:
# Confirming new number of columns
basics.shape

(360845, 9)

In [139]:
# Keeping start year 2000-2022
basics = basics[(basics['startYear'] >='2000') & (basics['startYear'] <= '2022')]
basics['startYear'].value_counts()

2017    14143
2018    14047
2016    13789
2019    13621
2015    13305
2014    12944
2013    12232
2012    11506
2021    11380
2020    11044
2011    10668
2010    10090
2009     9236
2008     8045
2007     6852
2006     6398
2005     5743
2004     5100
2022     4917
2003     4510
2002     4077
2001     3801
2000     3577
Name: startYear, dtype: int64

In [140]:
# Checking new dataframe shape
basics.shape

(211025, 9)

In [141]:
# Eliminating movies that include 'Documentary' in genre
docs_list = basics['genres'].str.contains('documentary', case=False)
basics = basics[~docs_list]
basics.shape

(139471, 9)

# Processing AKAs Data

In [ ]:
akas.

In [142]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32059183 entries, 0 to 32059182
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 1.9+ GB


In [143]:
keeps_list = basics['tconst'].isin(akas['titleId'])
keeps_list

34805      True
61119      True
67672      True
77968      True
86806      True
           ... 
8941011    True
8941020    True
8941059    True
8941104    True
8941188    True
Name: tconst, Length: 139471, dtype: bool

In [144]:
basics = basics[keeps_list]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77968,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86806,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
8941011,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
8941020,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,90,"Action,Adventure,Thriller"
8941059,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,NaN,84,Thriller
8941104,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"


In [145]:
akas = akas.replace({'\\N':np.nan})
akas.isna().sum()

titleId                   0
ordering                  0
title                     4
region              1927422
language            6199873
types              26740817
attributes         31819315
isOriginalTitle        2187
dtype: int64